pip install git+https://github.com/asimov-academy/Asimov_TV_connector.git

https://python.plainenglish.io/extracting-data-from-trading-view-253919ba7194

In [1]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime, timedelta
import pandas as pd

username = open('login_tv.txt', 'r').read()
password = open('passwd_tv.txt', 'r').read()

tv = TvDatafeed(username, password)

In [2]:
# tv.search_symbol('PETR4','BMFBOVESPA')
df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 5000)
df.head(3)

,symbol,open,high,low,close,volume
datetime,,,,,,
2025-01-30 16:35:00,BMFBOVESPA:PETR4,36.566880,36.586408,36.527823,36.547351,475410.814549
2025-01-30 16:40:00,BMFBOVESPA:PETR4,36.537587,36.605936,36.518059,36.586408,608038.346829
2025-01-30 16:45:00,BMFBOVESPA:PETR4,36.576644,36.586408,36.508294,36.527823,420618.745229


In [3]:
df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 10)
df

,symbol,open,high,low,close,volume
datetime,,,,,,
2025-04-23 16:40:00,BMFBOVESPA:PETR4,30.67,30.69,30.62,30.64,1103100.0
2025-04-23 16:45:00,BMFBOVESPA:PETR4,30.64,30.65,30.62,30.63,987900.0
2025-04-23 16:50:00,BMFBOVESPA:PETR4,30.63,30.68,30.50,30.58,2738200.0
2025-04-23 17:05:00,BMFBOVESPA:PETR4,30.57,30.57,30.57,30.57,5117800.0
2025-04-23 17:30:00,BMFBOVESPA:PETR4,30.57,30.65,30.57,30.60,148000.0
2025-04-23 17:35:00,BMFBOVESPA:PETR4,30.62,30.62,30.59,30.60,46700.0
2025-04-23 17:40:00,BMFBOVESPA:PETR4,30.60,30.62,30.58,30.58,15400.0
2025-04-23 17:45:00,BMFBOVESPA:PETR4,30.59,30.60,30.58,30.59,14400.0
2025-04-23 17:50:00,BMFBOVESPA:PETR4,30.59,30.60,30.56,30.57,16800.0


In [ ]:
url0 = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/'

url  = {'ibra50' :url+'Cart_IBr50.csv',
        'ibra100':url+'Cart_IBr100.csv',
        'idiv'   :url+'Cart_Idiv.csv',
        'ibov'   :url+'Cart_Ibov.csv',
        'small'  :url+'Cart_Small.csv'
}
indice = pd.read_csv(url['small'] ,
                     sep=';' , decimal=',' , thousands='.' ,
                     skiprows=1 , skipfooter=2 , encoding='latin-1',
                     index_col=False , engine='python')

indice

HTTPError: HTTP Error 404: Not Found

In [8]:
lista = ['PETR3','PETR4','VALE3','BBAS3']

In [9]:
for idx,row in enumerate(lista):
    i=row
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.to_csv("data/tvohlc.csv" , sep=';' , decimal='.')

In [10]:
def import_data(csv): 
    """ Renomeando as colunas para aritmetica com dataframe """
    def extrair_caracteres(s):
        import re
        pattern = re.compile(r"([^_]+)_")
    
        match = pattern.match(s)
        if match:
                return match.group(1)
        else:
             return None
    
    df = pd.read_csv(csv, sep=';' , decimal='.')
    df.index = pd.to_datetime(df['datetime'])#, format = "%y-%m-%d")
    df = df.drop('datetime', axis=1)
    
    close_columns    = []
    high_columns     = []
    low_columns      = []
    open_columns     = []
    volume_columns   = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    dicH = {s:extrair_caracteres(s) for s in high_columns}
    dicL = {s:extrair_caracteres(s) for s in low_columns}
    dicO = {s:extrair_caracteres(s) for s in open_columns}
    dicC = {s:extrair_caracteres(s) for s in close_columns}
    dicV = {s:extrair_caracteres(s) for s in volume_columns}
    
    close_df    = df[close_columns].rename(columns=dicC)
    high_df     = df[high_columns].rename(columns=dicH)
    low_df      = df[low_columns].rename(columns=dicL)
    open_df     = df[open_columns].rename(columns=dicO)
    volume_df   = df[volume_columns].rename(columns=dicV)
 
    return [close_df, high_df, low_df, open_df, volume_df]